## 랜덤 포레스트 모델 구축하기

### 학습 목표 
 * 라벨 인코딩을 수행한다. 
 * 데이터 EDA 및 시각화를 통해 데이터를 이해하고 기본 모델을 만들어본다.
 * 모델을 제출해 본다.(랜덤 포레스트 등)

* 데이콘 대회 : https://dacon.io/competitions/official/235745/overview/description
* 오류 관련 링크 : https://dacon.io/competitions/official/235745/talkboard/403708?page=1&dtype=recent

### 01. 데이터 불러오기 및 확인

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression

In [2]:
import pandas as pd

train = pd.read_csv("../data/parking_demand/train_df_errno.csv")
test = pd.read_csv("../data/parking_demand/test_df.csv")
sub = pd.read_csv("../data/parking_demand/sample_submission.csv")
age = pd.read_csv("../data/parking_demand/age_gender_info.csv")

train.shape, test.shape, sub.shape, age.shape

((2896, 15), (1008, 14), (150, 2), (16, 23))

In [3]:
train.columns

Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '10분내지하철수', '10분내버스정류장수', '단지내주차면수', '등록차량수'],
      dtype='object')

In [4]:
train.columns = ['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '10분내지하철수',
       '10분내버스정류장수', '단지내주차면수', '등록차량수']

test.columns = ['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '10분내지하철수',
       '10분내버스정류장수', '단지내주차면수']


### 02. 결측치를 처리(1)

### 데이터 결합

In [5]:
all_df = pd.concat([train, test], join='inner')
all_df

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수
0,C2515,545,아파트,경상남도,국민임대,33.48,276,17.0,A,9216000,82940,0.0,3.0,624.0
1,C2515,545,아파트,경상남도,국민임대,39.60,60,17.0,A,12672000,107130,0.0,3.0,624.0
2,C2515,545,아파트,경상남도,국민임대,39.60,20,17.0,A,12672000,107130,0.0,3.0,624.0
3,C2515,545,아파트,경상남도,국민임대,46.90,38,17.0,A,18433000,149760,0.0,3.0,624.0
4,C2515,545,아파트,경상남도,국민임대,46.90,19,17.0,A,18433000,149760,0.0,3.0,624.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,C1267,675,아파트,경상남도,행복주택,36.77,126,38.0,L,-,-,0.0,1.0,467.0
1004,C2189,382,아파트,전라북도,국민임대,29.19,96,45.0,H,6872000,106400,0.0,2.0,300.0
1005,C2189,382,아파트,전라북도,국민임대,29.19,20,45.0,H,6872000,106400,0.0,2.0,300.0
1006,C2189,382,아파트,전라북도,국민임대,39.45,202,45.0,H,13410000,144600,0.0,2.0,300.0


In [6]:
all_df.isnull().sum()

단지코드            0
총세대수            0
임대건물구분          0
지역              0
공급유형            0
전용면적            0
전용면적별세대수        0
공가수             0
자격유형            2
임대보증금         749
임대료           749
10분내지하철수      249
10분내버스정류장수      4
단지내주차면수         0
dtype: int64

###  자격유형(test) 결측치 처리

In [7]:
all_df['지역'].unique()

array(['경상남도', '대전광역시', '경기도', '전라북도', '강원도', '광주광역시', '충청남도', '부산광역시',
       '제주특별자치도', '울산광역시', '충청북도', '전라남도', '경상북도', '대구광역시', '서울특별시',
       '세종특별자치시'], dtype=object)

In [8]:
all_df.loc[all_df['자격유형'].isnull()]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수
196,C2411,962,아파트,경상남도,국민임대,46.90,240,25.0,NaN,71950000,37470,0.0,2.0,840.0
258,C2253,1161,아파트,강원도,영구임대,26.37,745,0.0,NaN,2249000,44770,0.0,2.0,173.0


In [9]:
grouped = all_df.groupby(['단지코드', '임대건물구분', '지역','공급유형'])
group1 = grouped.get_group( ('C2411', '아파트', '경상남도', '국민임대')  )
group1

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수
193,C2411,962,아파트,경상남도,국민임대,39.43,56,25.0,A,11992000,100720,0.0,2.0,840.0
194,C2411,962,아파트,경상남도,국민임대,39.72,336,25.0,A,11992000,100720,0.0,2.0,840.0
195,C2411,962,아파트,경상남도,국민임대,39.82,179,25.0,A,11992000,100720,0.0,2.0,840.0
196,C2411,962,아파트,경상남도,국민임대,46.90,240,25.0,NaN,71950000,37470,0.0,2.0,840.0
197,C2411,962,아파트,경상남도,국민임대,51.93,150,25.0,A,21586000,171480,0.0,2.0,840.0


In [10]:
group2 = grouped.get_group( ('C2253', '아파트', '강원도', '영구임대')  )
group2

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수
258,C2253,1161,아파트,강원도,영구임대,26.37,745,0.0,NaN,2249000,44770,0.0,2.0,173.0
259,C2253,1161,아파트,강원도,영구임대,31.32,239,0.0,C,3731000,83020,0.0,2.0,173.0
260,C2253,1161,아파트,강원도,영구임대,31.32,149,0.0,C,3731000,83020,0.0,2.0,173.0


In [11]:
all_df.loc[ 196, "자격유형"] = 'A'
all_df.loc[ 258, "자격유형"] = 'C'

In [12]:
print(all_df.자격유형.unique())

['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O']


In [13]:
mapping = { 'A':1, 'B':2, 'C':3, 'D':4, 'E':5, 
            'F':6, 'G':7, 'H':8, 'I':9, 'J':10, 
            'K':11, 'L':12, 'M':13, 'N':14, 'O':15  }

all_df['자격유형'] =all_df['자격유형'].map(mapping).astype(int)

In [14]:
print(all_df.공급유형.unique())

['국민임대' '공공임대(50년)' '영구임대' '임대상가' '공공임대(10년)' '공공임대(분납)' '장기전세' '공공분양'
 '행복주택' '공공임대(5년)']


In [15]:
all_df.columns

Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '10분내지하철수', '10분내버스정류장수', '단지내주차면수'],
      dtype='object')

### 10분내버스정류장수 (tr) 결측치 처리,

In [16]:
all_df.isnull().sum()

단지코드            0
총세대수            0
임대건물구분          0
지역              0
공급유형            0
전용면적            0
전용면적별세대수        0
공가수             0
자격유형            0
임대보증금         749
임대료           749
10분내지하철수      249
10분내버스정류장수      4
단지내주차면수         0
dtype: int64

In [17]:
all_df.loc[ train['10분내버스정류장수'].isnull(), :]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수
2293,N2431,1047,아파트,경상남도,공공임대(10년),74.97,80,15.0,1,46000000,456000,NaN,NaN,1066.0
2294,N2431,1047,아파트,경상남도,공공임대(10년),84.95,124,15.0,1,57000000,462000,NaN,NaN,1066.0
2295,N2431,1047,아파트,경상남도,공공임대(10년),84.96,289,15.0,1,57000000,462000,NaN,NaN,1066.0
2296,N2431,1047,아파트,경상남도,공공임대(10년),84.98,82,15.0,1,57000000,462000,NaN,NaN,1066.0


In [18]:
all_df['임대건물구분'].unique()

array(['아파트', '상가'], dtype=object)

In [19]:
pd.set_option("display.max_rows", 100)
pd.get_option("display.max_rows")

100

In [20]:
grouped = train.groupby(['임대건물구분', '지역'])
group1 = grouped.get_group( ('아파트', '경상남도')  )
group1

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수
0,C2515,545,아파트,경상남도,국민임대,33.48,276,17.0,A,9216000,82940,0.0,3.0,624.0,205.0
1,C2515,545,아파트,경상남도,국민임대,39.60,60,17.0,A,12672000,107130,0.0,3.0,624.0,205.0
2,C2515,545,아파트,경상남도,국민임대,39.60,20,17.0,A,12672000,107130,0.0,3.0,624.0,205.0
3,C2515,545,아파트,경상남도,국민임대,46.90,38,17.0,A,18433000,149760,0.0,3.0,624.0,205.0
4,C2515,545,아파트,경상남도,국민임대,46.90,19,17.0,A,18433000,149760,0.0,3.0,624.0,205.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2799,C2225,1206,아파트,경상남도,행복주택,26.17,120,49.0,K,51961000,27060,0.0,3.0,842.0,448.0
2800,C2225,1206,아파트,경상남도,행복주택,26.19,16,49.0,K,51961000,27060,0.0,3.0,842.0,448.0
2801,C2225,1206,아파트,경상남도,행복주택,36.03,290,49.0,K,72294000,37650,0.0,3.0,842.0,448.0
2802,C2225,1206,아파트,경상남도,행복주택,36.03,60,49.0,K,72294000,37650,0.0,3.0,842.0,448.0


In [21]:
# 데이터 확인 후, 임의 처리 4
all_df.loc[ all_df['10분내버스정류장수'].isnull(), "10분내버스정류장수"] = 4

In [22]:
all_df.loc[ all_df['10분내버스정류장수'].isnull(), :]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수


In [23]:
all_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3904 entries, 0 to 1007
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   단지코드        3904 non-null   object 
 1   총세대수        3904 non-null   int64  
 2   임대건물구분      3904 non-null   object 
 3   지역          3904 non-null   object 
 4   공급유형        3904 non-null   object 
 5   전용면적        3904 non-null   float64
 6   전용면적별세대수    3904 non-null   int64  
 7   공가수         3904 non-null   float64
 8   자격유형        3904 non-null   int32  
 9   임대보증금       3155 non-null   object 
 10  임대료         3155 non-null   object 
 11  10분내지하철수    3655 non-null   float64
 12  10분내버스정류장수  3904 non-null   float64
 13  단지내주차면수     3904 non-null   float64
dtypes: float64(5), int32(1), int64(2), object(6)
memory usage: 522.2+ KB


In [24]:
all_df.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수
0,C2515,545,아파트,경상남도,국민임대,33.48,276,17.0,1,9216000,82940,0.0,3.0,624.0
1,C2515,545,아파트,경상남도,국민임대,39.60,60,17.0,1,12672000,107130,0.0,3.0,624.0
2,C2515,545,아파트,경상남도,국민임대,39.60,20,17.0,1,12672000,107130,0.0,3.0,624.0
3,C2515,545,아파트,경상남도,국민임대,46.90,38,17.0,1,18433000,149760,0.0,3.0,624.0
4,C2515,545,아파트,경상남도,국민임대,46.90,19,17.0,1,18433000,149760,0.0,3.0,624.0


### 라벨 인코딩

In [25]:
for c in all_df.columns:
    print(all_df[c].unique())

['C2515' 'C1407' 'C1945' 'C1470' 'C1898' 'C1244' 'C1171' 'C2073' 'C2513'
 'C1936' 'C2049' 'C2202' 'C1925' 'C2576' 'C1312' 'C1874' 'C2650' 'C2416'
 'C2013' 'C1424' 'C2100' 'C2621' 'C2520' 'C2319' 'C1616' 'C1704' 'C2258'
 'C1032' 'C2038' 'C1859' 'C1722' 'C1850' 'C2190' 'C1476' 'C1077' 'C1068'
 'C1983' 'C2135' 'C2034' 'C1109' 'C1497' 'C2289' 'C2597' 'C2310' 'C1672'
 'C2132' 'C1439' 'C1613' 'C2216' 'C1899' 'C1056' 'C2644' 'C1206' 'C2481'
 'C1718' 'C1655' 'C1430' 'C1775' 'C1519' 'C2221' 'C1790' 'C2109' 'C1698'
 'C1866' 'C1005' 'C1004' 'C1875' 'C2156' 'C2212' 'C2401' 'C2571' 'C1175'
 'C1833' 'C2445' 'C1885' 'C2368' 'C2016' 'C2371' 'C2536' 'C2538' 'C1014'
 'C1592' 'C1867' 'C2326' 'C1015' 'C1620' 'C1049' 'C2000' 'C2097' 'C1668'
 'C1689' 'C1234' 'C2514' 'C1368' 'C1057' 'C2336' 'C1026' 'C2256' 'C1900'
 'C2666' 'C2361' 'C1642' 'C1013' 'C2232' 'C1973' 'C2458' 'C2574' 'C2133'
 'C2096' 'C2010' 'C1879' 'C1131' 'C1468' 'C1213' 'C1173' 'C2492' 'C2032'
 'C2094' 'C1880' 'C2089' 'C1744' 'C2046' 'C2071' 'C

In [26]:
gubun1 = {'아파트':1, '상가':2}
gubun2 = {'경상남도':1, '대전광역시':2, '경기도':3, '전라북도':4, '강원도':5,
          '광주광역시':6, '충청남도':7, '부산광역시':8, '제주특별자치도':9, '울산광역시':10,
          '충청북도':11, '전라남도':12, '경상북도':13, '대구광역시':14, '서울특별시':15, '세종특별자치시':16}

gubun3 = {'국민임대':1, '공공임대(50년)':2, '영구임대':3, '임대상가':4, '공공임대(10년)':5, 
          '공공임대(분납)':6, '장기전세':7, '공공분양':8, '행복주택':9, '공공임대(5년)':10}

all_df['임대건물구분_lbl'] = all_df['임대건물구분'].map(gubun1)
all_df['지역_lbl'] = all_df['지역'].map(gubun2)
all_df['공급유형_lbl'] = all_df['공급유형'].map(gubun3)

all_df

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,임대건물구분_lbl,지역_lbl,공급유형_lbl
0,C2515,545,아파트,경상남도,국민임대,33.48,276,17.0,1,9216000,82940,0.0,3.0,624.0,1,1,1
1,C2515,545,아파트,경상남도,국민임대,39.60,60,17.0,1,12672000,107130,0.0,3.0,624.0,1,1,1
2,C2515,545,아파트,경상남도,국민임대,39.60,20,17.0,1,12672000,107130,0.0,3.0,624.0,1,1,1
3,C2515,545,아파트,경상남도,국민임대,46.90,38,17.0,1,18433000,149760,0.0,3.0,624.0,1,1,1
4,C2515,545,아파트,경상남도,국민임대,46.90,19,17.0,1,18433000,149760,0.0,3.0,624.0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,C1267,675,아파트,경상남도,행복주택,36.77,126,38.0,12,-,-,0.0,1.0,467.0,1,1,9
1004,C2189,382,아파트,전라북도,국민임대,29.19,96,45.0,8,6872000,106400,0.0,2.0,300.0,1,4,1
1005,C2189,382,아파트,전라북도,국민임대,29.19,20,45.0,8,6872000,106400,0.0,2.0,300.0,1,4,1
1006,C2189,382,아파트,전라북도,국민임대,39.45,202,45.0,8,13410000,144600,0.0,2.0,300.0,1,4,1


In [27]:
all_df.단지코드.unique()

array(['C2515', 'C1407', 'C1945', 'C1470', 'C1898', 'C1244', 'C1171',
       'C2073', 'C2513', 'C1936', 'C2049', 'C2202', 'C1925', 'C2576',
       'C1312', 'C1874', 'C2650', 'C2416', 'C2013', 'C1424', 'C2100',
       'C2621', 'C2520', 'C2319', 'C1616', 'C1704', 'C2258', 'C1032',
       'C2038', 'C1859', 'C1722', 'C1850', 'C2190', 'C1476', 'C1077',
       'C1068', 'C1983', 'C2135', 'C2034', 'C1109', 'C1497', 'C2289',
       'C2597', 'C2310', 'C1672', 'C2132', 'C1439', 'C1613', 'C2216',
       'C1899', 'C1056', 'C2644', 'C1206', 'C2481', 'C1718', 'C1655',
       'C1430', 'C1775', 'C1519', 'C2221', 'C1790', 'C2109', 'C1698',
       'C1866', 'C1005', 'C1004', 'C1875', 'C2156', 'C2212', 'C2401',
       'C2571', 'C1175', 'C1833', 'C2445', 'C1885', 'C2368', 'C2016',
       'C2371', 'C2536', 'C2538', 'C1014', 'C1592', 'C1867', 'C2326',
       'C1015', 'C1620', 'C1049', 'C2000', 'C2097', 'C1668', 'C1689',
       'C1234', 'C2514', 'C1368', 'C1057', 'C2336', 'C1026', 'C2256',
       'C1900', 'C26

In [28]:
all_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3904 entries, 0 to 1007
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   단지코드        3904 non-null   object 
 1   총세대수        3904 non-null   int64  
 2   임대건물구분      3904 non-null   object 
 3   지역          3904 non-null   object 
 4   공급유형        3904 non-null   object 
 5   전용면적        3904 non-null   float64
 6   전용면적별세대수    3904 non-null   int64  
 7   공가수         3904 non-null   float64
 8   자격유형        3904 non-null   int32  
 9   임대보증금       3155 non-null   object 
 10  임대료         3155 non-null   object 
 11  10분내지하철수    3655 non-null   float64
 12  10분내버스정류장수  3904 non-null   float64
 13  단지내주차면수     3904 non-null   float64
 14  임대건물구분_lbl  3904 non-null   int64  
 15  지역_lbl      3904 non-null   int64  
 16  공급유형_lbl    3904 non-null   int64  
dtypes: float64(5), int32(1), int64(5), object(6)
memory usage: 613.8+ KB


In [29]:
all_df['단지코드'] = all_df['단지코드'].astype("category")

In [30]:
all_df['단지코드_lbl'] = all_df['단지코드'].cat.codes
all_df

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,임대건물구분_lbl,지역_lbl,공급유형_lbl,단지코드_lbl
0,C2515,545,아파트,경상남도,국민임대,33.48,276,17.0,1,9216000,82940,0.0,3.0,624.0,1,1,1,492
1,C2515,545,아파트,경상남도,국민임대,39.60,60,17.0,1,12672000,107130,0.0,3.0,624.0,1,1,1,492
2,C2515,545,아파트,경상남도,국민임대,39.60,20,17.0,1,12672000,107130,0.0,3.0,624.0,1,1,1,492
3,C2515,545,아파트,경상남도,국민임대,46.90,38,17.0,1,18433000,149760,0.0,3.0,624.0,1,1,1,492
4,C2515,545,아파트,경상남도,국민임대,46.90,19,17.0,1,18433000,149760,0.0,3.0,624.0,1,1,1,492
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,C1267,675,아파트,경상남도,행복주택,36.77,126,38.0,12,-,-,0.0,1.0,467.0,1,1,9,86
1004,C2189,382,아파트,전라북도,국민임대,29.19,96,45.0,8,6872000,106400,0.0,2.0,300.0,1,4,1,381
1005,C2189,382,아파트,전라북도,국민임대,29.19,20,45.0,8,6872000,106400,0.0,2.0,300.0,1,4,1,381
1006,C2189,382,아파트,전라북도,국민임대,39.45,202,45.0,8,13410000,144600,0.0,2.0,300.0,1,4,1,381


In [31]:
all_df_last = all_df.drop(['임대건물구분', '지역', '공급유형'] , axis=1)
all_df_last

,단지코드,총세대수,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,임대건물구분_lbl,지역_lbl,공급유형_lbl,단지코드_lbl
0,C2515,545,33.48,276,17.0,1,9216000,82940,0.0,3.0,624.0,1,1,1,492
1,C2515,545,39.60,60,17.0,1,12672000,107130,0.0,3.0,624.0,1,1,1,492
2,C2515,545,39.60,20,17.0,1,12672000,107130,0.0,3.0,624.0,1,1,1,492
3,C2515,545,46.90,38,17.0,1,18433000,149760,0.0,3.0,624.0,1,1,1,492
4,C2515,545,46.90,19,17.0,1,18433000,149760,0.0,3.0,624.0,1,1,1,492
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,C1267,675,36.77,126,38.0,12,-,-,0.0,1.0,467.0,1,1,9,86
1004,C2189,382,29.19,96,45.0,8,6872000,106400,0.0,2.0,300.0,1,4,1,381
1005,C2189,382,29.19,20,45.0,8,6872000,106400,0.0,2.0,300.0,1,4,1,381
1006,C2189,382,39.45,202,45.0,8,13410000,144600,0.0,2.0,300.0,1,4,1,381


In [32]:
for c in all_df_last.columns:
    print(all_df_last[c].unique())

['C2515', 'C1407', 'C1945', 'C1470', 'C1898', ..., 'C2456', 'C1266', 'C2152', 'C1267', 'C2189']
Length: 561
Categories (561, object): ['C2515', 'C1407', 'C1945', 'C1470', ..., 'C1266', 'C2152', 'C1267', 'C2189']
[ 545 1216  755  696  566 1722  624  361  754  240  688  623  601  405
  518  619  875  560  595  625  880 1396  970  606 1507  639  965  946
  785 2424  809  388 1013  453  861  806  903 1116 1486 1957  802 1527
 1005 1988 1350 2428 1755  840  390  451  410  779  693  753  498 1533
  711  420  590  657  495  460  338 1144  521 1003  306  697  213  481
  468 1364  800  830  775  261  474   72 1473  996  870  678  632  961
 1232  676 1300  998  493 1117  307  501  896  458  290  409  586 1084
 1174  270 1308  355  384  853  492  901  815  312  571  594  944  635
  962  822 1129 1479  330  386  456  642  302  757  705 1072  375 1018
  341  416  708  662 1002  462  781  496  630  512  534  762  890  494
  550  383  882  615  470  477 1260  773 1124  324 1497  531  389  712
  561  

In [33]:
train.shape, test.shape

((2896, 15), (1008, 14))

In [34]:
train_df = all_df_last.iloc[0:2896,:]
test_df = all_df_last.iloc[2896:,:]

train_df.shape, test_df.shape

train_df = pd.concat([train_df, train['등록차량수'] ], axis=1)
train_df

,단지코드,총세대수,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,임대건물구분_lbl,지역_lbl,공급유형_lbl,단지코드_lbl,등록차량수
0,C2515,545,33.48,276,17.0,1,9216000,82940,0.0,3.0,624.0,1,1,1,492,205.0
1,C2515,545,39.60,60,17.0,1,12672000,107130,0.0,3.0,624.0,1,1,1,492,205.0
2,C2515,545,39.60,20,17.0,1,12672000,107130,0.0,3.0,624.0,1,1,1,492,205.0
3,C2515,545,46.90,38,17.0,1,18433000,149760,0.0,3.0,624.0,1,1,1,492,205.0
4,C2515,545,46.90,19,17.0,1,18433000,149760,0.0,3.0,624.0,1,1,1,492,205.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2891,C2532,239,49.20,19,7.0,1,11346000,116090,0.0,1.0,166.0,1,5,1,501,146.0
2892,C2532,239,51.08,34,7.0,1,14005000,142310,0.0,1.0,166.0,1,5,1,501,146.0
2893,C2532,239,51.73,34,7.0,1,14005000,142310,0.0,1.0,166.0,1,5,1,501,146.0
2894,C2532,239,51.96,114,7.0,1,14005000,142310,0.0,1.0,166.0,1,5,1,501,146.0


In [35]:
train_df.columns

Index(['단지코드', '총세대수', '전용면적', '전용면적별세대수', '공가수', '자격유형', '임대보증금', '임대료',
       '10분내지하철수', '10분내버스정류장수', '단지내주차면수', '임대건물구분_lbl', '지역_lbl', '공급유형_lbl',
       '단지코드_lbl', '등록차량수'],
      dtype='object')

In [36]:
from sklearn.model_selection import train_test_split

In [37]:
sel = [ '총세대수', '전용면적', '전용면적별세대수', '공가수', '자격유형', '10분내버스정류장수', 
       '단지내주차면수', '임대건물구분_lbl', '지역_lbl', '공급유형_lbl',
       '단지코드_lbl']

X = train_df[sel]
y = train_df['등록차량수']
test_X = test_df[sel]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, 
                                                   random_state=0)

In [38]:
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [39]:
model = LinearRegression()
model.fit(X_train, y_train)
pred = model.predict(X_test)

print("학습(score) :", model.score(X_train, y_train) ) # 결정계수
print("테스트(score) :", model.score(X_test, y_test) ) # 결정계수

학습(score) : 0.7957809911189807
테스트(score) : 0.7988974846133388


In [40]:
mae_val = np.mean( abs( y_test - pred ) )
print( mae_val )
mse_val = np.mean( (y_test - pred) **2 )
print( mae_val )
rmse_val = mse_val ** 0.5
print( rmse_val )

142.33218947810167
142.33218947810167
201.661938023843


In [41]:
model = RandomForestRegressor(n_jobs=-1)
model.fit(X_train, y_train)
pred = model.predict(X_test)

print("학습(score) :", model.score(X_train, y_train) ) # 결정계수
print("테스트(score) :", model.score(X_test, y_test) ) # 결정계수

학습(score) : 0.9985243153096608
테스트(score) : 0.9936454071990892


In [42]:
mae_val = np.mean( abs( y_test - pred ) )
print( mae_val )
mse_val = np.mean( (y_test - pred) **2 )
print( mae_val )
rmse_val = mse_val ** 0.5
print( rmse_val )

16.419217491369388
16.419217491369388
35.84751444102563


In [43]:
model = RandomForestRegressor(n_jobs=-1)
model.fit(X_train, y_train)
pred = model.predict(test_X)
pred[0:10]

array([ 665.7 ,  708.52,  674.21,  674.21,  679.05,  680.74,  678.85,
        686.74, 1439.21, 1437.78])

In [44]:
test_df['등록차량수'] = pred
test_df['단지별차량수평균'] = test_df.groupby("단지코드")['등록차량수'].transform(np.mean)
test_new = test_df.drop_duplicates(['단지코드'], keep='first').reset_index()
test_new

<ipython-input-44-3cc158f3592c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['등록차량수'] = pred
<ipython-input-44-3cc158f3592c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['단지별차량수평균'] = test_df.groupby("단지코드")['등록차량수'].transform(np.mean)


,index,단지코드,총세대수,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,임대건물구분_lbl,지역_lbl,공급유형_lbl,단지코드_lbl,등록차량수,단지별차량수평균
0,0,C1072,754,39.79,116,14.0,8,22830000,189840,0.0,2.0,683.0,1,3,1,32,665.70,681.002500
1,8,C1128,1354,39.79,368,9.0,8,22830000,189840,0.0,3.0,1216.0,1,3,1,43,1439.21,1443.917778
2,17,C1456,619,33.40,82,18.0,1,19706000,156200,0.0,16.0,547.0,1,8,1,143,582.99,606.938889
3,26,C1840,593,39.57,253,7.0,1,14418000,108130,0.0,3.0,543.0,1,4,1,263,578.44,596.567500
4,30,C1332,1297,39.99,282,11.0,8,28598000,203050,0.0,2.0,1112.0,1,3,1,110,1013.36,1008.745000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,982,C2456,349,26.44,24,17.0,8,6992000,117000,0.0,4.0,270.0,1,9,1,471,218.30,215.472500
143,986,C1266,596,26.94,164,35.0,8,8084000,149910,0.0,1.0,593.0,1,11,1,85,507.77,512.142000
144,991,C2152,120,24.83,66,9.0,3,-,-,0.0,1.0,40.0,1,5,3,369,28.38,28.605000
145,993,C1267,675,24.87,28,38.0,8,6882000,104370,0.0,1.0,467.0,1,1,1,86,423.19,430.251818


In [45]:
add_dat = {'code':['C2675', 'C2335', 'C1327'],
           'num':['0', '0', '0']}
add_df = pd.DataFrame(add_dat)
add_df

,code,num
0,C2675,0
1,C2335,0
2,C1327,0


In [46]:
sub_df = test_new[ ['단지코드', '단지별차량수평균']]
sub_df.columns = ['code', 'num']
sub_df = pd.concat([sub_df, add_df]).reset_index()
sub_df = sub_df.drop(['index'], axis=1)
sub_df

,code,num
0,C1072,681.003
1,C1128,1443.92
2,C1456,606.939
3,C1840,596.567
4,C1332,1008.75
...,...,...
145,C1267,430.252
146,C2189,302.245
147,C2675,0
148,C2335,0


In [47]:
sub_df.to_csv('third_rf_0714.csv', index=False)
sub_df.head()

,code,num
0,C1072,681.003
1,C1128,1443.92
2,C1456,606.939
3,C1840,596.567
4,C1332,1008.75


In [48]:
import os
os.listdir(os.getcwd())

['.git',
 '.ipynb_checkpoints',
 '01_competition_firstmodel.html',
 '01_competition_firstmodel.ipynb',
 '01_competition_firstmodel.pdf',
 '01_competition_firstmodel_memo.pdf',
 '02_second_datapreprocessing.html',
 '02_second_datapreprocessing.ipynb',
 '02_second_datapreprocessing.pdf',
 '02_second_datapreprocessing_memo.pdf',
 '03_second_linear_model-Copy2.ipynb',
 '03_second_linear_model.html',
 '03_second_linear_model.ipynb',
 '03_second_linear_model.pdf',
 '03_second_linear_model_memo.pdf',
 '03_second_linear_ridge_lasso.ipynb',
 '04_second_rf_model.html',
 '04_second_rf_model.ipynb',
 '04_second_rf_model.pdf',
 '04_second_rf_model_memo.pdf',
 '05_second_etc_model.html',
 '05_second_etc_model.ipynb',
 '05_second_etc_model.pdf',
 'baseline_0712.csv',
 'fourth_rf_0714.csv',
 'README.md',
 'second_rf_0712.csv',
 'test_df.csv',
 'third_rf_0714.csv',
 'train_df.csv',
 'train_df_errno.csv',
 'Untitled.ipynb']

### 점수 : 138.65787